<h1>ChatBOTAgent</h1>

In [2]:
import pandas as pd
from fastapi.responses import JSONResponse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
# Đọc dữ liệu
# Đảm bảo đường dẫn file CSV đúng với môi trường thực tế
df = pd.read_csv('D:/UbuntuSystem/OnlineCodingCompetition/RecommendationSystem/dataset/goodreads_data.csv')  # Sửa lại đường dẫn nếu cần

df = df.dropna(subset=['Description'])  # Bỏ sách thiếu mô tả

# Tính toán độ tương tự giữa các sách
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Description'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['Book']).drop_duplicates()

In [4]:
def recommend_books(title:str, num_recommendations:int = 5):
    print(f"Received request to recommend books similar to: {title} with {num_recommendations} recommendations.")

    idx = indices.get(title)
    if idx is None:
        return JSONResponse(content={"error": f"Không tìm thấy sách '{title}' trong dữ liệu."}, status_code=404)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]
    book_indices = [i[0] for i in sim_scores]
    results = df.iloc[book_indices][['Book', 'Author', 'Genres', 'Description']].to_dict(orient='records')
    return {"recommendations": results}

In [5]:
def recommend_personalized_books(data: dict):
    data = data.json()
    user_id = data.get('user_id')
    num_recommendations = data.get('num_recommendations', 5)

    # Giả sử có một hàm để lấy lịch sử đọc sách của người dùng
    # history = get_user_reading_history(user_id)
    history = []  # Thay thế bằng hàm thực tế để lấy lịch sử đọc sách
    if not history:
        return JSONResponse(content={"error": "Không có lịch sử đọc sách cho người dùng."}, status_code=404)
    # Tính toán độ tương tự giữa sách trong lịch sử đọc
    history_indices = [indices.get(title) for title in history if indices.get(title) is not None]
    if not history_indices:
        return JSONResponse(content={"error": "Không tìm thấy sách trong lịch sử đọc."}, status_code=404)

    # Tính toán độ tương tự giữa sách trong lịch sử đọc
    history_cosine_sim = cosine_sim[history_indices]
    # Lấy trung bình các độ tương tự
    mean_sim_scores = history_cosine_sim.mean(axis=0)
    # Lấy top N sách tương tự
    book_indices = mean_sim_scores.argsort()[::-1][:num_recommendations]
    results = df.iloc[book_indices][['Book', 'Author', 'Genres', 'Description']].to_dict(orient='records')
    return {"recommendations": results}

In [10]:
import time
print("Book recommendation service is running...")
start_timing = time.time()
result = recommend_books(title="Harry Potter and the Philosopher’s Stone (Harry Potter, #1)", num_recommendations=10)
end_timing = time.time()
print(end_timing - start_timing, "seconds to recommend books")

Book recommendation service is running...
Received request to recommend books similar to: Harry Potter and the Philosopher’s Stone (Harry Potter, #1) with 10 recommendations.
0.018161296844482422 seconds to recommend books


In [11]:
for index, value in enumerate(result['recommendations']):
    print(f"{index + 1}. {value['Book']} - {value['Author']}")

1. Harry Potter and the Goblet of Fire (Harry Potter, #4) - J.K. Rowling
2. Harry Potter and the Chamber of Secrets (Harry Potter, #2) - J.K. Rowling
3. Harry Potter and the Prisoner of Azkaban (Harry Potter, #3) - J.K. Rowling
4. Harry Potter and the Order of the Phoenix (Harry Potter, #5) - J.K. Rowling
5. Harry Potter and the Cursed Child: Parts One and Two (Harry Potter, #8) - J.K. Rowling
6. Harry Potter and the Half-Blood Prince (Harry Potter, #6) - J.K. Rowling
7. Harry Potter and the Order of the Phoenix (Harry Potter, #5, Part 1) - J.K. Rowling
8. Quidditch Through the Ages - Kennilworthy Whisp
9. Harry Potter and the Deathly Hallows (Harry Potter, #7) - J.K. Rowling
10. The First Fifteen Lives of Harry August - Claire North


In [9]:
for index, value in enumerate(result['recommendations']):
    print(f"{index + 1}. {value['Book']} - {value['Author']}")

1. Pride and Prejudice and Zombies (Pride and Prejudice and Zombies, #1) - Seth Grahame-Smith
2. Austenland (Austenland, #1) - Shannon Hale
3. The Complete Novels - Jane Austen
4. Pride and Prejudice, Mansfield Park, Persuasion - Jane Austen
5. Dawn of the Dreadfuls (Pride and Prejudice and Zombies, #0.5) - Steve Hockensmith
6. Jane Austen: A Life - Claire Tomalin
7. Eligible: A Modern Retelling of Pride & Prejudice - Curtis Sittenfeld
8. Persuasion - Jane Austen
9. The Jane Austen Book Club - Karen Joy Fowler
10. Jane Eyre - Charlotte Brontë


In [7]:
for index, value in enumerate(result['recommendations']):
    print(f"{index + 1}. {value['Book']} - {value['Author']}")

1. Go Set a Watchman - Harper Lee
2. Harper Lee's To Kill a Mockingbird - Harold Bloom
3. Return to the Castle (Castle of Mysteries #2) - Julie-Anne Gamble
4. The Chronicles of Prydain (The Chronicles of Prydain #1-5) - Lloyd Alexander
5. Discovery of a Hidden Castle (A Gypsy Curse #1) - Julie-Anne Gamble
6. Castle of Mysteries Three: The Complete Series - Julie-Anne Gamble
7. Dreamscape - Rose Anderson
8. Mockingbird - Walter Tevis
9. Amazing Grief A Healing Guide for Parents of Young Addicts - Charles Harper
10. The 5 Love Languages: The Secret to Love that Lasts - Gary Chapman
